In [ ]:
import numpy as np
import cv2
import os
from imutils import paths
import pickle

### Initializing

In [ ]:
training_images = "dataset/training"
testing_images = "dataset/testing"

In [ ]:
# The folders are defined as 0 and 1, due to them needing to be integers for the 
# model required by opencv LBPH classifier
# So, we'll define the database here
training_database = ["person_a", "person_b"]
testing_database = {'2': 'person_a', '3': 'person_a'}

In [ ]:
def face_detection(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     print(image_gray)
    haar_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    face = haar_classifier.detectMultiScale(image_gray, scaleFactor=1.3, minNeighbors=7)
    if len(face) == 0:
        return None, None
    (x,y,w,h) = face[0]
    return image_gray[y:y+w, x:x+h], face[0]

### Extracting features

In [ ]:
def prepare_data(data_path):
    folders = os.listdir(data_path)
    labels = []
    faces = []
    for folder in folders:
        label = int(folder)
        training_images_path = data_path + '/' + folder
        for image in os.listdir(training_images_path):
            image_path = training_images_path + '/' + image
            training_image = cv2.imread(image_path)
            face, bounding_box = face_detection(training_image)
            if face is not None:
                faces.append(face)
                labels.append(label)

    print ('Training Done')
    return faces, labels

In [ ]:
faces, labels = prepare_data(training_images)
print ('Total faces = ', len(faces))
print ('Total labels = ', len(labels))

In [ ]:
model = cv2.face.LBPHFaceRecognizer_create()
model.train(faces, np.array(labels))

Number of features
* Using uniform patterns, the length of the feature vector for a single cell reduces from 256 to 59. The 58 uniform binary patterns correspond to the integers 0, 1, 2, 3, 4, 6, 7, 8, 12, 14, 15, 16, 24, 28, 30, 31, 32, 48, 56, 60, 62, 63, 64, 96, 112, 120, 124, 126, 127, 128, 129, 131, 135, 143, 159, 191, 192, 193, 195, 199, 207, 223, 224, 225, 227, 231, 239, 240, 241, 243, 247, 248, 249, 251, 252, 253, 254 and 255

In [ ]:
# Saving the model
filename = "lbph_model.yml"
model.write(filename)

### Face Recognition

In [ ]:
# Loading the model
model = cv2.face.LBPHFaceRecognizer_create()
filename = "lbph_model.yml"
model.read(filename)

In [ ]:
def predict_image(test_image):
    img = test_image.copy()
    face, bounding_box = face_detection(img)
    if face is not None:
        label = model.predict(face)
        label_text = training_database[label[0]]
        return label_text
    else:
        return None

In [ ]:
prediction_status = {'correct': 0, 'incorrect': 0, 'unidentified': 0}

In [ ]:
for imagePath in paths.list_images(testing_images):
    # load the image, convert it to grayscale, describe it,
    # and classify it
    test = cv2.imread(imagePath)
    predict = predict_image(test)
    actual = testing_database[imagePath.split(os.path.sep)[-2]]
    print(f"Actual label: {actual}, Predicted label: {predict}")
    if predict == actual:
        prediction_status['correct'] += 1
    elif predict == None:
        prediction_status['unidentified'] += 1
    elif predict != actual:
        prediction_status['incorrect'] += 1

In [ ]:
print(prediction_status)